















































































































































































































# Generate NDD CSVs













































## Setup -- You need a spark session! 
(but the default/smallest one should be fine) 

In [ ]:
#setup - packages & env
import pyspark
import dxdata
import dxpy
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
#setup - grabbing dataset - make sure to add most recent dataset here
dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]

In [ ]:
# the participant dataset is the one we want to work with 
dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]

# List of ICD10 codes / UKB Field IDs

#see spreadsheet here: https://docs.google.com/document/d/1AebkQ-Nxrk63jhsDzZpn5QD-7EK4unsykHVj-saEm3U/edit?usp=sharing for more info

In [ ]:
# Pull down the fields we need 
field_names = ["eid", "p31", "p34", "p22189", "p22006", "p21022", 'p42028', 'p42020', 'p42018', 'p131042', 'p42032', 'p42022', 'p42024', 'p22009_a1', 'p22009_a2', 'p22009_a3', 'p22009_a4', 'p22009_a5', 'p40000_i0']
df = participant.retrieve_fields(names=field_names, coding_values="replace", engine=dxdata.connect())

In [ ]:
# Send to Pandas 
df = df.toPandas()

In [ ]:
# Human readable columns please
df = df.rename(columns={'eid':'ID',
                'p31':'GENETIC_SEX', 
                'p34':'BIRTH_YEAR', 
                'p22189':'TOWNSEND', 
                'p22006':'ETHNICITY', 
                'p21022':'AGE_OF_RECRUIT',
                "p42028":'ALS_DATE',       
                'p42020':'AD_DATE',
                'p42018':'DEM_DATE',  
                'p131042': 'MS_DATE',
                'p42032':'PD_DATE',
                'p42022':'VAS_DATE',
                'p42024':'FTD_DATE',
                'p22009_a1':'PC1',
                'p22009_a2':'PC2',
                'p22009_a3':'PC3',
                'p22009_a4':'PC4',
                'p22009_a5':'PC5',
                'p40000_i0':'date_of_death'
                                             
               })

df

In [ ]:
date = 'SEPT_2023'
ndd_list = ['ALS', 'AD', 'DEM', 'MS', 'PD', 'VAS', 'FTD']

for ndd in ndd_list:
    test = df[~df[f'{ndd}_DATE'].isna()]
    test = test[['ID', 'GENETIC_SEX', 'BIRTH_YEAR', 'TOWNSEND', 'ETHNICITY',
       'AGE_OF_RECRUIT', ndd + '_DATE', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5',
       'date_of_death']]
    a = len(test)
    print(ndd, a)
    test.to_csv(f'{ndd}_cases_n{a}_{date}.csv', header = True, index = False)

In [ ]:
import pandas as pd
test = pd.read_csv('ALS_cases_n753_SEPT_2023.csv')
test

# Save all files

In [ ]:
for ndd in ndd_list:
    test = df[~df[f'{ndd}_DATE'].isna()]
    a = len(test)
    print(f'dx upload {ndd}_cases_n{a}_{date}.csv --path /data/NDD_cases/{ndd}_cases_n{a}_{date}.csv')

In [ ]:
%%bash
dx upload ALS_cases_n753_SEPT_2023.csv --path /data/NDD_cases/ALS_cases_n753_SEPT_2023.csv
dx upload AD_cases_n4447_SEPT_2023.csv --path /data/NDD_cases/AD_cases_n4447_SEPT_2023.csv
dx upload DEM_cases_n10043_SEPT_2023.csv --path /data/NDD_cases/DEM_cases_n10043_SEPT_2023.csv
dx upload MS_cases_n2595_SEPT_2023.csv --path /data/NDD_cases/MS_cases_n2595_SEPT_2023.csv
dx upload PD_cases_n4413_SEPT_2023.csv --path /data/NDD_cases/PD_cases_n4413_SEPT_2023.csv
dx upload VAS_cases_n2182_SEPT_2023.csv --path /data/NDD_cases/VAS_cases_n2182_SEPT_2023.csv
dx upload FTD_cases_n319_SEPT_2023.csv --path /data/NDD_cases/FTD_cases_n319_SEPT_2023.csv